<a href="https://colab.research.google.com/github/Ordo-Umbra/Void-Defect-Model/blob/main/vdmenginesmanalog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import eigh

# (Paste the full VDMEngine class from before here—unchanged base)

class VDMEngineSM(VDMEngine):  # Inherits base; add SM flavor
    def __init__(self, *args, quantize_spawns=True, **kwargs):
        super().__init__(*args, **kwargs)
        self.quantize_spawns = quantize_spawns
        self.bound_evals = self.bound_state_spectra(V0=7.0, sigma=2.0)  # Tune for levels
        self.bound_masses = np.abs(self.bound_evals)  # Positive for E/mass proxy
        print(f"SM Bound levels (neg evals): {self.bound_evals}")
        print(f"SM Mass proxies (abs): {self.bound_masses}")

    def tick(self):
        """Overridden tick: Full copy with quantization in spawns."""
        # Emergent sigma
        dist_matrix = np.linalg.norm(self.positions[:, np.newaxis] - self.positions, axis=2)
        sigma = np.mean(dist_matrix[dist_matrix > 0]) / np.sqrt(2) if self.N > 1 else 1.0

        grad, kernel, diffs, Phi = self.compute_Phi_and_grad(self.positions, self.E, sigma)
        force_mags = np.linalg.norm(grad, axis=1)  # FIXED: Define here

        # Spawn logic
        spawn_threshold = np.mean(Phi)
        spawn_prob_base = self.chaos_lambda * np.std(force_mags) if np.std(force_mags) > 0 else 0.406
        spawns = []
        for i in range(self.N):
            if force_mags[i] > spawn_threshold and self.rng.random() < spawn_prob_base * (force_mags[i] / spawn_threshold):
                closest_j = np.argmin(dist_matrix[i] + 1e6 * (np.arange(self.N) == i))
                mid_point = (self.positions[i] + self.positions[closest_j]) / 2
                new_pos1 = mid_point + self.rng.normal(0, 0.1, self.dims)
                new_pos2 = mid_point + self.rng.normal(0, 0.1, self.dims)
                new_vel1 = self.rng.uniform(-0.1, 0.1, self.dims)
                new_vel2 = self.rng.uniform(-0.1, 0.1, self.dims)
                parent_E_half = self.E[i] / 2
                if self.quantize_spawns:
                    # FIXED: Snap to closest |E_n| (positive mass)
                    dist_to_levels = np.abs(self.bound_masses - parent_E_half)
                    closest_idx = np.argmin(dist_to_levels)
                    new_E = self.bound_masses[closest_idx]
                    print(f"Quantized spawn: parent {self.E[i]:.1f} -> level {closest_idx} E={new_E:.2f}")
                else:
                    new_E = parent_E_half
                new_S = self.S[i] / 2
                spawns.append((new_pos1[None, :], new_pos2[None, :], new_vel1[None, :], new_vel2[None, :],
                               new_E, new_E, new_S[None, :], new_S[None, :]))

        # Apply spawns (rest as base)
        for p1, p2, v1, v2, e1, e2, s1, s2 in spawns:
            if self.N >= self.max_N:
                break
            self.positions = np.vstack([self.positions, p1, p2])
            self.velocities = np.vstack([self.velocities, v1, v2])
            self.E = np.append(self.E, [e1, e2])
            self.S = np.vstack([self.S, s1, s2])
            self.N += 2

        # Recompute post-spawn
        dist_matrix = np.linalg.norm(self.positions[:, np.newaxis] - self.positions, axis=2)
        sigma = np.mean(dist_matrix[dist_matrix > 0]) / np.sqrt(2) if self.N > 1 else 1.0
        grad, kernel, diffs, Phi = self.compute_Phi_and_grad(self.positions, self.E, sigma)

        # Stall mask
        Phi_crit = np.max(Phi) / 2 if np.max(Phi) > 0 else 15.0
        stall_mask = Phi > Phi_crit
        self.velocities[stall_mask] *= 0.1

        # Spin update (torque=0 for central forces)
        spin_dims = self.S.shape[1]
        if self.dims == 2:
            S_scalar = self.S[:, 0].copy()
            torque = np.zeros(self.N)
            S_scalar += self.dt * torque / self.E
            self.S[:, 0] = S_scalar
            # Tangential grad for 2D
            perp_diffs = np.stack([-diffs[:,:,1], diffs[:,:,0]], axis=2)  # (N,N,2)
            tang_force = S_scalar[:, np.newaxis, np.newaxis] * perp_diffs / (dist_matrix[:,:,np.newaxis]**2 + 1e-8)
            tang_grad = np.sum(tang_force, axis=1)  # (N,2)
            grad += tang_grad
        else:  # 3D
            torque = np.zeros((self.N, spin_dims))
            self.S += self.dt * torque / self.E[:, np.newaxis]
            # Tangential grad
            tang_force = np.cross(self.S[:, np.newaxis, :], diffs) / (dist_matrix[:,:,np.newaxis]**2 + 1e-8)  # (N,N,3)
            tang_grad = np.sum(tang_force, axis=1)  # (N,3)
            grad += tang_grad

        # Update dynamics
        self.velocities = self.damping * self.velocities - self.dt * grad
        self.velocities += self.chaos_lambda * self.rng.normal(0, 0.05, self.velocities.shape)
        self.positions += self.dt * self.velocities
        self.pos_history.append(self.positions.copy())

        # Energy evolution
        alpha = 0.01 / len(self.pos_history)  # Approx t+1
        interact_mask = kernel > 0.5
        self.E += alpha * np.sum(interact_mask, axis=1)

        I_avg = np.mean(np.exp(-dist_matrix**2 / sigma**2))
        if I_avg > self.threshold_I:
            print(f"Cohesion at tick {len(self.pos_history)-1}")

    def plot_E_histogram(self):
        """Histogram final E; peaks at discrete masses."""
        plt.figure(figsize=(8,5))
        plt.hist(self.E, bins=20, alpha=0.7, edgecolor='black')
        for idx, level in enumerate(self.bound_masses):
            plt.axvline(level, color='red', linestyle='--', label=f'Gen {idx}')
        plt.title('Emergent Mass Spectrum (E Histogram)')
        plt.xlabel('Energy / Mass Proxy')
        plt.ylabel('Count')
        plt.legend()
        plt.show()

# Example SM Run (paste after classes)
# engine_sm = VDMEngineSM(dims=2, N_initial=50, steps=300, max_N=500, quantize_spawns=True)
# engine_sm.run()
# engine_sm.plot_trajectories()
# engine_sm.plot_E_histogram()

In [ ]:
engine_sm = VDMEngineSM(dims=2, N_initial=100, steps=300, max_N=500, quantize_spawns=True)
engine_sm.run()
engine_sm.plot_trajectories()
engine_sm.plot_E_histogram()

Streaming output truncated to the last 5000 lines.
Quantized spawn: parent 18.0 -> level 0 E=6.11
Quantized spawn: parent 11.9 -> level 0 E=6.11
Quantized spawn: parent 10.6 -> level 0 E=6.11
Quantized spawn: parent 17.9 -> level 0 E=6.11
Quantized spawn: parent 15.6 -> level 0 E=6.11
Quantized spawn: parent 19.2 -> level 0 E=6.11
Quantized spawn: parent 18.1 -> level 0 E=6.11
Quantized spawn: parent 17.4 -> level 0 E=6.11
Quantized spawn: parent 17.4 -> level 0 E=6.11
Quantized spawn: parent 13.6 -> level 0 E=6.11
Quantized spawn: parent 15.6 -> level 0 E=6.11
Quantized spawn: parent 15.7 -> level 0 E=6.11
Quantized spawn: parent 15.2 -> level 0 E=6.11
Quantized spawn: parent 11.3 -> level 0 E=6.11
Quantized spawn: parent 13.3 -> level 0 E=6.11
Quantized spawn: parent 12.9 -> level 0 E=6.11
Quantized spawn: parent 16.1 -> level 0 E=6.11
Quantized spawn: parent 17.9 -> level 0 E=6.11
Quantized spawn: parent 18.1 -> level 0 E=6.11
Quantized spawn: parent 13.4 -> level 0 E=6.11
Quantized